Ok, now let's make a model. The paper I'm modelling this project on uses a modified version of Structural Deep Neighbor Embedding (SDNE): https://www.kdd.org/kdd2016/papers/files/rfp0191-wangAemb.pdf. There's a very nice implementation of SDNE in keras (along with some other embeddings) here: https://github.com/palash1992/GEM. Instead of creating SDNE from scratch, I'm just going to modify the SDNE model in the GEM package.

The SDNE model in the GEM package takes one large graph and trains on batches of its edges. There are a few ways we could extend this to our problem- we could pad each team's graph with isolated nodes and feed each team's graph in one at a time, or we could create one big graph formed of several disjoint connected-component graphs consisting of each team's roster. The latter makes more sense here- we want to learn the embedding of each individual player node in the training set and how the properties of each node can be used to predict unobserved edges, not necessarily a set of graph properties that can be generalized across teams (which is a harder problem). In other words, for now, I'm interpolating, not extrapolating. For generalizable properties, we'd likely need to look at stuff like GCNs or GAT models- things that learn bigger structural properties.

There is one small assumption that goes into this- that two blockers with the same name are the same player. Derby names are punny, so it's not rare that a good player name like 'Smackpropagation' will show up on a few different teams. However, the nature of our pruning algorithm tends to only give us data for graphs for decently established teams, and this name-similarity problem tends to only show up with more local teams (who are less likely to have enough stats on the website).

The Teammate algorithm randomly removes 20% of the edges from the network, sets this aside as a test network and the non-removed 80% as a train network. We need to do this too.

Because we are only learning observed edges, not null ones, and the random train-test subsampling procedure described in the Team Composition paper can create disconnected graphs that are then fed into the autoencoder, it doesn't seem like strongly-connectedness is necessarily important for the Teammate algorithm to work (just that it's considered good form). Spectral-decomposition based algorithms do require connectivity, but I don't believe this one does.

The Teammate paper uses random-walk sampling on the test set. Because our graph is made of several disjoint components rather than one large LCC, I'm also not entirely convinced this step will do anything for us, so I'll leave it out for now.




-Train on the train network. The loss function should be the same as SDNE i.e. (L = $\sum_{i=1}^{n} |(\hat{x}_{i}-x_{i})  \odot [b_{(i,j)}]_{j=1}^{n}|^{2}$) but with the follwing change:

Base SDNE: $b_{(i,j)} = 1$ if $(i,j)= 0, b_{(i,j)} = \beta > 1$ otherwise.

Teammate Encoder SDNE: $b_{(i,j)} = 0$ if $(i,j)= 0, b_{(i,j)} = 1$ otherwise.

In standard SDNE, the loss function penalizes zero edges. However, we don't just want to penalize them- we want to completely ignore them. We're not trying to predict *if* players have played together, just how well they do play together when they do.




-Test on the test network. The paper